# INBOX

See all incoming messages in this notebook. Everyone can log in with their access data and assign this notebook to their HMI. To do this, simply enter your own HMI with the corresponding secret as the client in this script. Running this notebook, you will authorize your HMI to call up the wheel loader's location. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [ ]:
import s3i
import time 
import base64 
import getpass
import receiver
import pykka
import jwt
import json
import logging
logging.basicConfig(level=logging.ERROR)

This Notebook acts as a client in terms of the S³I. In order to use the S³I this notebook needs a client id and the respective secret. You can assign this notebook to your personal HMI, to make this notebook your HMI. Therefore enter the id and the secret of your HMI in the following input fields.

In [ ]:
clientId = input('[S3I]: Please enter the client id:')
clientSecret = getpass.getpass('[S3I]: Please enter the secret:')
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: Client id and secret are set")

Next, you have to enter your username and password. With your access data a token is requested which authorizes this client (your HMI) to call up the wheel loader's location on your behalf.

In [ ]:
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) +
          "]: DEMO Wheel loader (service call), please log in! (sachverstaendiger)")
username = input('[S3I]: Please enter your username:')
password = getpass.getpass('[S3I]: Please enter the password:')
print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()
                                   ) + "]: Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                         client_id=clientId, client_secret=clientSecret, username=username, password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN, scope="")

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                               time.localtime()) + "]: Token received " + parsed_username + " logged in.")

In [ ]:
class MessageDecryptor(pykka.ThreadingActor):
       
    def callback(self, pgp_message):
        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: A new message has been received")
        message_str = pgp_message.replace("\\n", "\n").strip('"') 
        uMsg = s3i.UserMessage(msg_blob=message_str)
    
        access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
        dir = s3i.Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
        """Decrypt the message and verify the signature
        """
        if uMsg.pgpMsg.is_encrypted:
            sec_key = s3i.Key(path_demo_dir="key/", filename="secKey.asc")
            uMsg.decryptAndVerify(sec_key, "", dir)
            #uMsg.decrypt(sec_key.key)
            uMsg.convertPgpToMsg()

        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Subject of the message: " + uMsg.msg["subject"])
        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Text of the message:  " + uMsg.msg["text"])
        print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: Sender of the message:  :", dir.queryThingIDBased(uMsg.msg["sender"]+"/attributes/name"))

        attachments_list = uMsg.msg["attachments"]
        """
        store the attachment file in specified path
        """
        for attachment in attachments_list:
            with open("received_data/"+attachment["filename"], 'wb') as file:
                decode = base64.b64decode(attachment["data"])
                file.write(decode)
                print("[S3I]: Attachment " + attachment["filename"]
                   + " of the message is stored in received_data")
                            
        print("==============================================================================")

                            
messageDecryptor = MessageDecryptor.start().proxy()

In [ ]:
s3i_receiver = receiver.Receiver.start("s3ibs://"+clientId,s3i_identity_provider,messageDecryptor).proxy()
s3i_receiver.start_receiving()